In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import json
import tqdm
import itertools
import inspect
from functools import partial
import random


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize

In [50]:
# from load_data import load_tasks, get_functions
from arc.load_data import load_tasks, get_functions

def plot_task(task):
    """ plots a task """
    examples = task['train']
    n_examples = len(examples)
    cmap = ListedColormap([
        '#000', '#0074D9', '#FF4136', '#2ECC40', '#FFDC00',
        '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'
    ])
    norm = Normalize(vmin=0, vmax=9)
    figure, axes = plt.subplots(2, n_examples, figsize=(n_examples * 4, 8))
    for column, example in enumerate(examples):
        axes[0, column].imshow(example['input'], cmap=cmap, norm=norm)
        axes[1, column].imshow(example['output'], cmap=cmap, norm=norm)
        axes[0, column].axis('off')
        axes[1, column].axis('off')
    plt.show()

tasks_train = load_tasks('training')
tasks_test = load_tasks('evaluation')
plot_task(tasks_train['0a938d79'])

ImportError: cannot import name 'get_functions' from 'arc.load_data' (/Users/hilalmufti/programs/arc-experiments/arc/load_data.py)

In [ ]:
# from _search import search, seq_to_program, i2p
from arc.arc_search._search import search, i2p
import arc.arc_dsl.dsl as dsl

MAX_LENGTH = 40

DSL_primitives = get_functions(dsl)
itop = [p.__name__ for p in DSL_primitives]
ptoi = {p: i for i, p in enumerate(itop)}

i_identity = ptoi['identity']

i2p = partial(i2p, itop=itop)
i2p = np.vectorize(i2p)

primitive_names = {p.__name__ for p in DSL_primitives}
print(f"DSL consists of {len(DSL_primitives)} primitives: {primitive_names}")

In [ ]:
primitive_names

In [ ]:
# the maximum composition depth to consider
MAX_DEPTH = 2

# construct the program strings of all programs expressible by composing at most MAX_DEPTH primitives
program_strings = search(MAX_DEPTH, primitive_names)
# print some of the program strings
print('\n'.join([*program_strings[:10], '...']))
print(f'Space to search consists of {len(program_strings)} programs:\n')

# map program strings to programs
programs = {prog_str: eval(prog_str) for prog_str in program_strings}

In [ ]:
# TODO: How do I paralellize this
def solve(tasks, programs):
    guesses = dict()
    for (key, task) in tqdm.tqdm(tasks.items()):
        train_inputs = [example['input'] for example in task['train']]
        train_outputs = [example['output'] for example in task['train']]
        hypotheses = []
        # iterate over all programs
        if isinstance(programs, dict):
            programs = programs.items()
        for program_string, program in programs:
            try:
                if all([program(i) == o for i, o in zip(train_inputs, train_outputs)]):
                    # remember program if it explains all training examples
                    hypotheses.append(program_string)
            except:
                pass
        # select first program for making predictions
        if len(hypotheses) > 0:
            print(f'found {len(hypotheses)} candidate programs for task {key}!')
            guesses[key] = hypotheses[0]
    print(f'\nMade guesses for {len(guesses)} tasks')
    return guesses

guesses = solve(tasks_train, programs)
guesses

In [ ]:
# make predictions and evaluate them

solved = dict()

# iterate over all tasks for which a guess exists
for key, program_string in guesses.items():
    # test_inputs = [example['input'] for example in train_challenges[key]['test']]
    test_inputs = [example['input'] for example in tasks_train[key]['test']]
    test_outputs = [example['output'] for example in tasks_train[key]['test']]
    program = eval(program_string)
    if all([program(i) == o for (i, o) in zip(test_inputs, test_outputs)]):
    # if all([program(i) == o for i, o in zip(test_inputs, train_solutions[key])]):
        # mark predition as correct if all test examples are solved by the program
        solved[key] = program_string


print(f'Predictions correct for {len(solved)}/{len(guesses)} tasks')

In [ ]:
# visualize solved tasks
for key, program_string in solved.items():
    print(f'For task "{key}", found program "{program_string}"')
    plot_task(tasks_train[key])

While this code runs extremely fast (considering 400 tasks in less than 10 seconds), it also performs extremely poor (solving only 4/400 tasks, giving a mere 1% accuracy). How could it be improved? What can be learned from it? There are at least two issues with the above approach to searching a space of programs expressible in a DSL: First, the DSL provided here is not very expressive in that for most tasks, no program that solves it exists in the set of all possible programs buildable in the DSL. Second, even if sufficient expressivity is guaranteed (e.g. via a turing-complete DSL), for which by definition there would exist solution programs to each task, such programs may in practice either not be discoverable in the first place or not detectable as correct programs.

### Increasing DSL Expressivity
It is obvious why some tasks can't be solved by the above DSL, but one simple proof would be the following: No primitive ever increases the pixel count of a grid, hence neither can any composition of the primitives ever do so - and since for certain tasks, the output grids do have more pixels than the input grids, the DSL is incomplete. To increase the expressibity of the program space (disregarding the maximum program size), one will want to expand the set of the primitives and also extend the structure of the considered programs beyond mere composition. Maybe it is a good idea to have primitives which take more than one input argument, or primitives that operate on types other than only grids, such as objects or integers. Note that viewing the transformations from inputs to outputs as a linear function composition is very misleading, as many tasks can't be neatly squeezed into this form: Some tasks seem much better addressed on a pixel- or object-level than on a grid-level. A good DSL is probably concise and allows expressing solutions to many tasks as short programs. Such a DSL may best be built by bootstrapping, that is, building a minimal version of it and then iterating back and forth between using it to solve ARC tasks and expanding it to account for unsolvable ARC tasks, all while having abstractness and flexibility of the primitives and how they can interplay in mind.

In [ ]:
# example of task where above DSL is ill-suited and e.g. an object-centric view could be more appropriate.
# Potentially useful primitives: object extraction, property detection (has 6 pixels) and transformations (recolor)
plot_task(tasks_train['b775ac94'])
# this guy needs a much simpler dsl

In [ ]:
# example of task where above DSL is ill-suited and e.g. a more pixel-centric view could be more appropriate.
# Potentially useful: cellular automata
plot_task(tasks_train['3906de3d'])

### Increasing Search Efficiency
To increase the efficiency of the search for programs, it is crucial to avoid a brute force search, as it very quickly becomes entirely infeasible as the size of the programs and the number of primitives grow: Even for a simplistic DSL where the primitives can only interact with each other via composition, if it consists of p primitives and one wants to check all programs up to depth d, the number of programs to consider with grow on the order of $p^d$.

Naively, "pruning the search space" or avoiding considering primitives or subprograms unlikely useful for a solution program could be addressed by building heuristics into the search, e.g. "whenever the output grid is always of the same size as the input grid for all training examples, don't consider grid rescaling operations during the search". However, not only would such an approach be extremely labour-intensive, but also likely prone to result in a brittle and static system that would performs poorly. Maybe the navigation of the search space is something that can or even should be learned, not hardcoded.

In [ ]:
# some calculations on infeasibility of brute force search
pd_arr = [(p, d) for p in [8, 16, 32] for d in [2, 4, 8]]
for p, d in sorted(pd_arr, key=lambda pd: pd[0]**pd[1]):
    print(f'DSL with {p} primitives and max. depth {d} allows > {p**d} programs')

### Outlook

Creating your own DSL from scratch will probably not be easy and may not be desired or needed in the first place. There are at least two open-source DSLs for ARC, for example [Johan's DSL](https://github.com/top-quarks/ARC-solution) which he also used to win the first ARC-competition in 2020 with 20.6% by having efficient implementations and simple primitive signatures rendering a large-scale search feasible. Alternatively, I have built a separate, less minimal [ARC-DSL](https://github.com/michaelhodel/arc-dsl) that comes alongside reference [solution programs for all 400 ARC training tasks](https://github.com/michaelhodel/arc-dsl/blob/main/solvers.py) and which was also used for more recent program synthesis approaches such as [CodeIt](https://arxiv.org/pdf/2402.04858) that may be useful, even if just as inspiration. However, not following other's footsteps can also have benefits, such as not having to work with something that may be suboptimal for the desired approach at hand and by that getting stuck in a local minimum, and maybe a more adequate DSL for ARC is yet to be created, maybe by you!

And, of course, using a DSL and searching over it is by no means the only way or all there is to ARC. Arguably, much work is yet to be done, and novelty and diversity in approaches seem in dire need to make progress on the benchmark, and with that hopefully ultimaltely towards much more advanced AI.

Happy ARC-ing and good luck with the ARC-prize!

In [ ]:
# let's try to make a submission

submission = dict()
# iterate over all tasks
for key, task in tqdm.tqdm(tasks_train.items()):
    train_inputs = [example['input'] for example in task['train']]
    train_outputs = [example['output'] for example in task['train']]
    hypotheses = []
    # iterate over all programs
    for program_string, program in programs.items():
        try:
            if all([program(i) == o for i, o in zip(train_inputs, train_outputs)]):
                # remember program if it explains all training examples
                hypotheses.append(program_string)
        except:
            pass
    # select first program for making predictions
    predictions = [example['input'] for example in task['test']]
    if len(hypotheses) > 0:
        print(f'found {len(hypotheses)} candidate programs for task {key}!')
        program_string = hypotheses[0]
        program = eval(program_string)
        try:
            predictions = [program(example['input']) for example in task['test']]
        except:
            pass
    submission[key] = [{'attempt_1': grid, 'attempt_2': grid} for grid in predictions]
print(f'\nMade guesses for {len(guesses)} tasks')

with open('submission.json', 'w') as fp:
    json.dump(submission, fp)

In [33]:
from arc.arc_search._mcts import *

In [38]:
print_abstract_program(('hello', 'world'))

(>>> hello world)


In [39]:
from arc.load_data import load_tasks

In [41]:
next(iter(load_tasks('training').items()))

{'a85d4709': {'train': [{'input': ((0, 0, 5), (0, 5, 0), (5, 0, 0)),
    'output': ((3, 3, 3), (4, 4, 4), (2, 2, 2))},
   {'input': ((0, 0, 5), (0, 0, 5), (0, 0, 5)),
    'output': ((3, 3, 3), (3, 3, 3), (3, 3, 3))},
   {'input': ((5, 0, 0), (0, 5, 0), (5, 0, 0)),
    'output': ((2, 2, 2), (4, 4, 4), (2, 2, 2))},
   {'input': ((0, 5, 0), (0, 0, 5), (0, 5, 0)),
    'output': ((4, 4, 4), (3, 3, 3), (4, 4, 4))}],
  'test': ({'input': ((0, 0, 5), (5, 0, 0), (0, 5, 0)),
    'output': ((3, 3, 3), (2, 2, 2), (4, 4, 4))},)},
 'c8cbb738': {'train': [{'input': ((3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3),
     (3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3),
     (3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3),
     (3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3),
     (3, 3, 1, 3, 3, 3, 1, 4, 3, 4, 3),
     (3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3),
     (3, 3, 3, 3, 1, 3, 2, 3, 3, 3, 2),
     (3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3),
     (8, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3),
     (3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3),
     (8, 3, 3, 3, 8, 3, 2, 3, 3, 3, 2),
     (3

In [24]:
make_node(())

Node(state=(), visits=0, value=0.0, parent=None, children=set())

In [18]:
make_foo

<function __main__.make_foo(bar, baz)>

In [21]:
make_foo('one', 'two')

['foo', 'one', 'two']

In [ ]:
make_program_string(('hello', 'world', 'ayo'))

'lambda I: ayo(world(hello(I)))'

In [15]:
n = Node(())
c = Node(('A'))

node_append(n, c)
n

Node(state=(), visits=0, value=0.0, parent=None, children=[Node(state='A', visits=0, value=0.0, parent=None, children=[...]), Node(state='A', visits=0, value=0.0, parent=None, children=[...])])

In [4]:
root = make_toy_tree()

root

Node(state=(), visits=0, value=0.0, parent=None, children=[Node(state=('A',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=None), children=[Node(state=('A', 'C'), visits=0, value=0.0, parent=Node(state=('A',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=None), children=None), children=None), Node(state=('A', 'D'), visits=0, value=0.0, parent=Node(state=('A',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=None), children=[Node(state=('A', 'C'), visits=0, value=0.0, parent=Node(state=('A',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=None), children=None), children=None)]), children=None)]), Node(state=('B',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=[Node(state=('A',), visits=0, value=0.0, parent=Node(state=(), visits=0, value=0.0, parent=None, children=None), children=None)]), 

In [51]:
show_tree(root)

() n=0 v=0.00
  ('A',) n=0 v=0.00
    ('A', 'C') n=0 v=0.00
    ('A', 'D') n=0 v=0.00
  ('B',) n=0 v=0.00
    ('B', 'E') n=0 v=0.00
    ('B', 'F') n=0 v=0.00
    ('B', 'G') n=0 v=0.00


In [15]:
state, parent, cs, _, _ = root
cs[('A',)]

(('A',),
 ((),
  None,
  {('A',): (...),
   ('B',): (('B',),
    (...),
    {('B', 'E'): (('B', 'E'), (...), {}, 0, 0.0),
     ('B', 'F'): (('B', 'F'), (...), {}, 0, 0.0),
     ('B', 'G'): (('B', 'G'), (...), {}, 0, 0.0)},
    0,
    0.0)},
  0,
  0.0),
 {('A', 'C'): (('A', 'C'), (...), {}, 0, 0.0),
  ('A', 'D'): (('A', 'D'), (...), {}, 0, 0.0)},
 0,
 0.0)

In [47]:
n = make_node(())
show_node(n)

() n=0 v=0.00


In [40]:
node_visit(n)

7

In [9]:
show_tree(root)

() n=0 v=0.00
  ('A',) n=0 v=0.00
    ('A', 'C') n=0 v=0.00
    ('A', 'D') n=0 v=0.00
  ('B',) n=0 v=0.00
    ('B', 'E') n=0 v=0.00
    ('B', 'F') n=0 v=0.00
    ('B', 'G') n=0 v=0.00


In [51]:
('hello',) + ('world',)

('hello', 'world')

Current sum: 0, Player 0's turn


ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: traced array with shape int32[]
The problem arose with the `int` function. If trying to convert the data type of a value, try using `x.astype(int)` or `jnp.array(x, int)` instead.
The error occurred while tracing the function select_action at /var/folders/4k/1j9dmqmj48n0ly5bvj__zrc80000gn/T/ipykernel_80220/4120847378.py:29 for jit. This concrete value was not available in Python because it depends on the values of the arguments node.visit_count, node.children[1].visit_count, node.children[1].total_value, node.children[2].visit_count, and node.children[2].total_value.

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError